# Сессия 4

## 4.1 Разработка бота

In [1]:
import pickle
import re
import pandas as pd
import scipy.sparse
import pymorphy2

In [2]:
class bot:
    
    def __init__(self):
        # Преобразователь для приведения слов текста в нормальную форму
        self.morph = pymorphy2.MorphAnalyzer(lang='ru')
        
        # Преобразователь текста в вид разряженной матрицы
        self.text_trans = pickle.load(open('text_transformer.pkl', 'rb'))
        
        # Список лишних атрибутов в разряженной матрице
        self.attrs_to_del = pickle.load(open('attrs_to_delS3.pkl', 'rb'))
        
        # Сама модель которая используется для получения адресата по тексту письма
        self.model = pickle.load(open('model.pkl', 'rb'))
        
        # Список ключевых слов нужный для обработки естественной речи 
        self.commands_key_words = {
                '/help' : '''помощь помогите справка узнать команды может привет здравствуй'''.split(),
                '/exit' : '''пока прощай пошёл ухожу выйти выход'''.split(),
                '/model_predict' : '''направить письмо заявление узнать адресат написано'''.split(),
            }
        for key in self.commands_key_words.keys():
            self.commands_key_words[key] = self.lemmatization(self.commands_key_words[key])
    
    # Очистка текста дял естественной речи 
    def input_preprocess(self, text):
        text = text.lower()
        text = re.sub('\W', ' ', text)
        text = re.sub(' +', ' ', text)
        return text
    
    # Лемматизация слов списка слов
    def lemmatization(self, word_list):
        lem_list = []
        for word in word_list:
            lem_list.append(self.morph.parse(word)[0].normal_form)
        return lem_list
    
    # Метод для получения предсказаний на основе списка текстов, возвращает список, 
    # где в каждом элементе находится список из классов и их вероятности 
    def model_predict(self, text_col):
        text_col = pd.Series(text_col).str.lower()\
                            .str.replace('[^a-zа-я ]', '', regex=True)\
                            .str.split()\
                            .apply(lambda x: ' '.join(self.lemmatization(x)))
        
        data_to_predict = pd.DataFrame.sparse.from_spmatrix(
                                self.text_trans.transform(text_col),
                                columns=self.text_trans.get_feature_names_out())\
                            .drop(columns=self.attrs_to_del)
        
        return [list(zip(self.model.classes_, probas)) for probas in self.model.predict_proba(data_to_predict)]
    
    # Вывод текста ботом
    def bot_print(self, text):
        print(f'Бот: {text}\nПользователь: ', end='')
    
    # Получения справки комманд бота
    def help_me(self):
        self.bot_print('''Мои команды:
/help - увидеть это сообщение
/bot_predict - получить адресата по тексту
/exit - выйти из бота
Также бот поддерживает естественный язык, попробуйте написать "Кому бы можно было направить письмо, в котором написано о квалификации на повышение"'''
        )
    
    # Получение предсказаний ботом
    def bot_predict(self, text = -1, mode = -1):
        if text == -1:
            self.bot_print('Введите ваше сообщение для получения адресата')
            text = str(input())
    
        if mode == -1:
            self.bot_print('В каком режиме вы хотите получить адресат[а/ов] сообщения 0 - один наиболее вероятный, 1 - несколько вероятных')
            mode = bool(int(input()))
            
        to_predict = [text]
        prediction = sorted(self.model_predict(to_predict)[0], key = lambda x: x[1], reverse=True)
        if mode == 0:
            self.bot_print(f'Наиболее вероятным адресатом вашего сообщения является - {prediction[0][0]}')
        elif mode == 1:
            text_output = '\n'.join([f'{x[0]} - {x[1]}' for x in prediction if x[1] != 0])
            self.bot_print(f'Наиболее вероятными адресатами вашего сообщения могут являться - {text_output}')
            
    
    # Получение комманды бота на основе списка нормализованных слов
    def get_command(self, norm_text_list):
        command = ['None', 0]
        for key in self.commands_key_words.keys():
            index = 0
            for word in norm_text_list:
                if word in self.commands_key_words[key]:
                    index += 1
            if index > command[1]:
                command[0] = key
                command[1] = index
        return command[0]
    
    # Обработчик естественного языка ботом
    def NL_process(self, text):
        full_text = text
        text = self.input_preprocess(text[:30])
        norm_text_list = self.lemmatization(text.split())
        norm_text = ' '.join(norm_text_list)
        command = self.get_command(norm_text_list)
        
        if command == '/help':
            self.help_me()
        elif command == '/exit':
            return 'Exit'
        elif command == '/model_predict':
            self.bot_predict(full_text)
        else:
            self.bot_print('Cообщение не распознано')
     

In [3]:
# Функция для начала работы бота в консоли 
def bot_start_pooling():
    bot_ = bot()
    bot_.bot_print('Для получения справки команд напишите /help, или можете также делать запрос на естественном языке')
    while True:
        command = str(input())
        if command == '/help':
            bot_.help_me()
        elif command == '/bot_predict':
            bot_.bot_predict()
        elif command == '/exit':
            break
        else:
            return_ = bot_.NL_process(command)
            if return_ == 'Exit':
                break

In [4]:
# Позволяет как запускать бота в консоли, так и импортировать его функции в другой код
if __name__ == '__main__':
    bot_start_pooling()

Бот: Для получения справки команд напишите /help, или можете также делать запрос на естественном языке
Пользователь: asdfasdf
Бот: Cообщение не распознано
Пользователь: 

KeyboardInterrupt: Interrupted by user